# V. Create sending address

In the previous sections Alice identified a transaction destined for her, and extracted all information she could from the transaction. She is currently the owner of 69300000 piconeros that was received in one prior output. Alice now wants to spend all of that Monero by sending it to Bob. She **only** knows Bob's Monero address.

Using only Bob's Monero address, Alice will create transaction data Bob will use to identify the Monero she sends to him. This section essentialy creates the opposite side of some of Alice's prior calculations. This section will cover all of the calculations Alice uses to create address data for Bob. The section after this one will cover Alice preparing amounts to sendd.

Alice eventually uses the data calculated in this section and following sections to spend her Monero in transaction txn_9e29.json. The data calculated in this section will be compared to data in that transaction to clarify that the calculations here are in fact exactly what is contained within that transaction.

There are two primary goal for this section:
1. generate transaction data Bob will use to identify Monero destined to him
2. generate transaction data for the left over Monero Alice sends back to herself (it will be zero), this step will be explained

This section relies heavliy on the following source:
* [Zero to Monero: Second Edition; Chapter 4](https://www.getmonero.org/library/Zero-to-Monero-2-0-0.pdf)

**It is highly recommended that the reader read chapter 4 of _Zero to Monero: Second Edition_ prior to reading the following sections.**

The sections below cover:
1. converting Bob's address to public spend/view keys
2. calculating transaction key (r), and transaction public key (rG)
3. calculating stealth addresses, and view tag
4. encrypting payment id
5. creating transaction extra field

### Summary (Introduction)
**What we start with:**
* Bob's Monero address: `49FgswTBtGdTZQPj7e16qc4jrWY5VuGE2aDoaMDEALknZeTy5vcjkcsCrh4deo231LKcACcjgALFji4c4SYQF5WdNCLvkN3`
* Alice's public view key: `b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867`
* Alice's public spend key: `cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37`

**What we end up with:**
* Bob's public view key
* Bob's public spend key
* transaction key (r)
* transaction public key (rG)
* stealth address (one for Bob, one for change) 
* view tag (one for Bob, one for change)
* encrypted payment id
* transaction extra field

## 1. Converting Bob's address to public spend/view keys

This was essentially already covered back in step 7 of 1_address_calculation.ipynb! Alice will extract the public spend / view keys from Bob's Monero address.

In [1]:
'''
Zero to Monero: Second Edition; Section 4.1
'''


'''
To calculate the public view key this code performs the following:

  1. Defines the base58 "alphabet"
  2. Iterates over the Monero address
  3. Converts 11 base58 characters at a time to an integer
  4. Converts the integer to big-endian byte array
  5. Converts the byte array to hex
  6. Concatenates all of the create hex strings
  7. Extracts the net byte + public spend key + public view key + checksum
'''

import math


# define a function to convert base58 encoding to hex encoding
def base58_to_hex(encoded_str):
    # Define the base58 alphabet and initialize some variables
    alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    base58_alphabet_size = len(alphabet)
    num = 0
    count = 0
    result = ''

    # Iterate over each character in the encoded string
    for i, char in enumerate(encoded_str):
        # Find the index of the current character in the alphabet
        base58_index = alphabet.index(char)
 
        # Multiply the current result by the base and add the current index
        num = num * base58_alphabet_size + base58_index
        count += 1

        # If we've processed 11 characters or reached the end of the string,
        # convert the result to a big-endian byte array and then to hex
        if count == 11 or i == len(encoded_str) - 1:
            
            # Convert the byte array to a hex string and add it to the result
            bits_in_number = num.bit_length()
            bytes_in_number = math.ceil(bits_in_number / 8.0)
            number_as_hex = num.to_bytes(bytes_in_number, byteorder="big").hex()
            print(f"Inside function - base58 block to integer: {num} bytes in number: {bytes_in_number} number as hex: {number_as_hex}")
            result += number_as_hex
            
            # Reset the variables for the next block
            num = 0
            count = 0

    return result


# convert Bob's Monero address to hex
bob_address_hex = base58_to_hex("49FgswTBtGdTZQPj7e16qc4jrWY5VuGE2aDoaMDEALknZeTy5vcjkcsCrh4deo231LKcACcjgALFji4c4SYQF5WdNCLvkN3")

# extract the first two characters for the net byte
netbyte_recovered = bob_address_hex[:2]

# extract 64 characters (32 bytes) for the public spend key
public_spend_key_bob = bob_address_hex[2:66]

# extract 64 characters (32 bytes) for the public view key 
public_view_key_bob = bob_address_hex[66:130]

# extract the checksum
checksum_recovered = bob_address_hex[130:]

print(f"Bob's public spend key (recovered from address) is: {public_spend_key_bob}")
print(f"Bob's public view key (recovered from address) is: {public_view_key_bob}")

Inside function - base58 block to integer: 1353714824316208806 bytes in number: 8 number as hex: 12c95c75de8c46a6
Inside function - base58 block to integer: 11441590088826347079 bytes in number: 8 number as hex: 9ec8b190ce1fde47
Inside function - base58 block to integer: 1610714593390301103 bytes in number: 8 number as hex: 165a686385910faf
Inside function - base58 block to integer: 14311635267191758287 bytes in number: 8 number as hex: c69d2472ffeaddcf
Inside function - base58 block to integer: 14064011468851997708 bytes in number: 8 number as hex: c32d67eaa5f16c0c
Inside function - base58 block to integer: 5107932096636181507 bytes in number: 8 number as hex: 46e3052dccd0dc03
Inside function - base58 block to integer: 8015621996052569946 bytes in number: 8 number as hex: 6f3d35bca0855f5a
Inside function - base58 block to integer: 17689745263234606342 bytes in number: 8 number as hex: f57e9997b63f6506
Inside function - base58 block to integer: 806887968104 bytes in number: 5 number as

## 2. Calculating transaction key (r), and transaction public key (rG)

The transaction key (r) is a random number generated by the wallet software when sending a transaction. The value `r` is **not** stored in the blockchain. The value is only stored by the wallet, and is viewable as the "transaction key" within wallet software. The transaction public key that is used in creating stealth addresses is simply `r` multiplied by the generator point `G`.

In [2]:
'''
Zero to Monero: Second Edition; Section 4.2
'''


'''
To calculate transaction public key this code performs the following:

  1. Defines the transaction key (r) previously generated by a Monero wallet
  2. Multiplies the transaction key by the generator point (G)
'''

import nacl.bindings

from binascii import hexlify, unhexlify


# define a function to multiply a scalar by the base point G
scalar_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp


# generate a random number to use as the transaction key (previously generated by a Monero wallet)
r = "42adc98ec7a15451f3a21750b2172e14d851191a7c07d247591bbd063b698a02"

# multiply "r" by G to create the transaction public key
rG = scalar_mult_G(unhexlify(r))


print(f"Transaction key (r): {r}")
print(f"Transaction pubic key (rG): {hexlify(rG).decode()}")

Transaction key (r): 42adc98ec7a15451f3a21750b2172e14d851191a7c07d247591bbd063b698a02
Transaction pubic key (rG): 0286418eb6ae7e07927819e3ab0bfb9aede712ca02352522e9d0f081df3d873d


## 3. Calculating stealth addresses, and view tag

Alice previously performed most of the following calculations when determining if she was the intended recipient of a transaction. She will generate two addresses: one address for Bob, and one address for return change. In this instance Alice is sending all of her Monero to Bob so her change should be zero, which it is. The second address is used to send return change of 0 Monero, but instead of sending a zero Monero output to Alice, a zero Monero output is send to a random address. The order in which the two outputs are represented in the transaction is randomly decided to further obfuscate transactions. In this case the zero change sent to a random address is output 0 and the spend to Bob is output 1. 

To further clarify the concept of "change" image if Alice had 3 Monero to spend and wanted to send 2 Monero, she would receive 1 Monero back as change. In this case Alice is spending all of her Monero and would not receive change. But Monero transactions always have at least two outputs to help ensure that the number of transaction outputs doesn't identify types of spending habits.

The first two sections below create the stealth address and view tag for the output destined to Bob. The second two sections create the stealth address and view tag for the zero amount change output destined to a random address.

In [3]:
'''
Zero to Monero: Second Edition; Section 4.2
'''


'''
To calculate a stealth address this code performs the following:

  1. Multiplies Bob's public view key by the transaction public key
  2. Concatenates the value created in step 1 and an index identifying 
     the output number (uses zero indexing)
  3. Hashes this value with Keccack
  4. Multiplies this by G to create a new curve point
  5. Adds Bob's public spend key to the curve point from step 4
'''

import varint
import nacl.bindings

from Cryptodome.Hash import keccak


# define a function to return a hashed byte string
def keccak_256(data):
    return keccak.new(digest_bits=256).update(data).digest()

# define a function to convert an integer into a valid ed25519 scalar
def pad_and_reduce(scalar: bytes) -> bytes:
    point_padded_to_64_bytes = scalar + (64 - len(scalar)) * b"\0"
    return nacl.bindings.crypto_core_ed25519_scalar_reduce(point_padded_to_64_bytes)


# define a function to multiply the ed25519 generator point G by a scalar (eg: integer)
scalar_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp

# define a function to multiply a curve point by a scalar
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp

# define a function to add to curve points
point_add = nacl.bindings.crypto_core_ed25519_add

# Monero multiplies curve points by 8, create 8 in bytes for use in calculations
cofactor = int(8).to_bytes(32, byteorder="little")


# multiply Bob's public view key by the transaction key to obtain rKv: rKv = r * G
rKv = scalar_point_mult(unhexlify(r), unhexlify(public_view_key_bob))

# multiply by 8, required by Monero
rKv = scalar_point_mult(cofactor, rKv)

# concatenate rKv and an integer index to indicate this output is the second output (zero indexing)
rKv_concat_extra_index = rKv + varint.encode(1)

# hash the previously concatenated value to create H(rKv)
HrKv = keccak_256(rKv_concat_extra_index)

# multiply the scalar by the ed25519 generator point to obtain a new curve point
HrKvG = scalar_mult_G(HrKv)

# add Bob's public spend key to the above value:  stealth address = H(rKv)G + public_spend_key
stealth_address_bob = point_add(HrKvG, unhexlify(public_spend_key_bob))


print(f"Stealth address for Bob (Output 1): {hexlify(stealth_address_bob).decode()}")

Stealth address for Bob (Output 1): 07ab8e6d3981d84b50f0d02ec45c2232ba239159db2f15adf44947535be13967


In [4]:
'''
Not present in  Zero to Monero: Second Edition
'''


'''
To calculate the view tag this code performs the following:

  1. Concatenates the string "view_tag" with the value rKv created for
     the stealth address in the prior section and an index identifying the 
     output number (uses zero indexing)
  2. Hashes this value with Keccack
  3. Extracts the first byte from this hash
'''

# concatenate the string "view_tag", rKv, and a transaction index integer
rKv_concat_extra_index = b"view_tag" + rKv + varint.encode(1)

# hash the previously concatenated value to create H(rKv)
HrKv = keccak_256(rKv_concat_extra_index)

# the view tag is the first byte of this hashed string
view_tag_bob = HrKv[0:1]


print(f"View tag for Bob (Output 1): {hexlify(view_tag_bob).decode()}")


View tag for Bob (Output 1): c6


The process of creating a random address to receive the zero amount output starts similar to normal change that is actually destined for Alice. But instead of adding Alice's public spend key, the calculation adds a randomly generated public spend key. To show the process of creating this stealth address the calculations "back into" the randomly generated public spend key by using the first stealth address from the actual transaction data and subtracting off an interediate step. That is obviously not the standard process (subtracting off a value) but is included for clarity to show the randomly generated public spend key.

In [5]:
'''
Zero to Monero: Second Edition; Section 4.2
'''


'''
To calculate a stealth address this code performs the following:

  1. Multiplies Alice's public view key by the transaction public key
  2. Concatenates the value created in step 1 and an index identifying 
     the output number (uses zero indexing)
  3. Hashes this value with Keccack
  4. Multiplies this by G to create a new curve point
  **** Begin: Normally not required
  5. Uses the actual send to identify the real stealth address calculated
  6. Subtracts the value in step 4 from the value in step 5 to get the fake public spend key
  **** End: Normally not required
  7. Adds the public spend key from step 6 to the curve point from step 4
'''

public_view_key_alice = "b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867"
public_spend_key_alice = "cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37"
stealth_address_0 = "4e9451dd54125562c76cb59decc500b396fb2f6dc73f95876adaa6729c7db3f8"

# define a function to subtract to curve points
point_sub = nacl.bindings.crypto_core_ed25519_sub


# multiply Alice's public view key by the transaction key to obtain rKv: rKv = r * G
rKv = scalar_point_mult(unhexlify(r), unhexlify(public_view_key_alice))

# multiply by 8, required by Monero
rKv = scalar_point_mult(cofactor, rKv)

# concatenate rKv and an integer index to indicate this output is the first output (zero indexing)
rKv_concat_extra_index = rKv + varint.encode(0)

# hash the previously concatenated value to create H(rKv)
HrKv = keccak_256(rKv_concat_extra_index)

# multiply the scalar by the ed25519 generator point to obtain a new curve point
HrKvG = scalar_mult_G(HrKv)

# intermediate step to obtain the randomly generated zero amount public spend key
public_spend_key_random = point_sub(unhexlify(stealth_address_0), HrKvG)

# recalculate the stealth address for clarity: stealth address  = H(rKv)G + public_spend_key
stealth_address_zero_amount = point_add(HrKvG, public_spend_key_random)


print(f"Stealth address for zero amount (Output 0): {hexlify(stealth_address_zero_amount).decode()}")


Stealth address for zero amount (Output 0): 4e9451dd54125562c76cb59decc500b396fb2f6dc73f95876adaa6729c7db3f8


In [6]:
'''
Not present in  Zero to Monero: Second Edition
'''


'''
To calculate the view tag this code performs the following:

  1. Concatenates the string "view_tag" with the value rKv created for
     the stealth address in the prior section and an index identifying the 
     output number (uses zero indexing)
  2. Hashes this value with Keccack
  3. Extracts the first byte from this hash
'''

# concatenate the string "view_tag", rKv, and a transaction index integer
rKv_concat_extra_index = b"view_tag" + rKv + varint.encode(0)

# hash the previously concatenated value to create H(rKv)
HrKv = keccak_256(rKv_concat_extra_index)

# the view tag is the first byte of this hashed string
view_tag_zero_amount = HrKv[0:1]


print(f"View tag for zero amount (Output 0): {hexlify(view_tag_zero_amount).decode()}")


View tag for zero amount (Output 0): bf


## 4. Encrypting payment id

All transactions include a payment id. If the sender doesn't choose one then the payment id is set to 0 and then encrypted. The encrypted payment id is created as though it were meant for Bob and not the zero amount change output.

In [7]:
'''
Zero to Monero: Second Edition; Section 4.4 (but applied to regular addresses not integrated addresses)
'''


'''
To create the payment id this code performs the following:

  1. Multiplied Bob's public view key by the transaction key
  2. Concatenates the value rKv and a Monero constant (the integer 141 as bytes)
  3. Creates an obfuscation mask from the value in step 1
  4. Created a zero payment id 8 bytes in length
  5. Truncates the obfuscation mask to 8 bytes
  6. Encrypts the payment id by applying bitwise xor operation with the value from step 2
  7. Displays the encrypted payment id as a list of byte values displayed as integers
'''

# define a function to calculate exclusive or between bits in two variables
def calculate_xor(var1, var2):
    return bytearray(a ^ b for a, b in zip(*map(bytearray, (var1, var2))))


# multiply Alice's public view key by the transaction key to obtain rKv: rKv = r * G
rKv = scalar_point_mult(unhexlify(r), unhexlify(public_view_key_bob))

# multiply by 8, required by Monero
rKv = scalar_point_mult(cofactor, rKv)

# convert the number 141 to bytes 
encrypted_payment_id_tail = int(141).to_bytes()

# create obfuscation mask
payment_id_mask = rKv + encrypted_payment_id_tail
payment_id_mask = keccak_256(payment_id_mask)

# payment id of zeros
payment_id = int(0).to_bytes(8)

# truncate obfuscation mask to length of encrypted payment id
truncated_payment_id_mask = payment_id_mask[: len(payment_id)]

# apply binary xor operation to encrypted payment id
encrypted_payment_id = calculate_xor(truncated_payment_id_mask, payment_id)


print(f"Payment id: {hexlify(payment_id).decode()}")
print(f"Encrypted payment id: {hexlify(encrypted_payment_id).decode()}")
print(f"Encrypted payment id (list form for extra field): {list(encrypted_payment_id)}")

Payment id: 0000000000000000
Encrypted payment id: 462650609010339e
Encrypted payment id (list form for extra field): [70, 38, 80, 96, 144, 16, 51, 158]


## 5. Creating transaction extra field

The transaction public key and encrypted payment id do not have their own defined field. Instead they are stored in the `extra` section of a transaction. The `extra` section can contain arbitrary data so special identifiers are used to "flag" specific data. The transaction public key is the first entry in `extra` and is prepended with the integer `1`. The encrypted payment id is the second entry in `extra` and is prepended with the integer `2` to indicate it is an encrypted payment id, and the integer `9` to indicate that there will be 9 bytes following the number `2`. This is all presented as a list of integers corresponding to the byte values of the original data.

In [8]:
'''
Zero to Monero: Second Edition; Section 4.4 (but applied to regular addresses not integrated addresses)
'''


'''
To create the payment id this code performs the following:

  1. Multiplied Bob's public view key by the transaction key
  2. Concatenates the value rKv and a Monero constant (the integer 141 as bytes)
  3. Creates an obfuscation mask from the value in step 1
  4. Created a zero payment id 8 bytes in length
  5. Truncates the obfuscation mask to 8 bytes
  6. Encrypts the payment id by applying bitwise xor operation with the value from step 2
  7. Displays the encrypted payment id as a list of byte values displayed as integers
'''

# include a transaction public key identifier
extra = [1]

# convert the transaction public key to a list of integers and add to extra
extra.extend(list(rG))

# include an encrypted payment id identifier
extra.append(2)

# include an identifer for the number of integers to follow including this one
extra.append(9)

# convert the encrypted payment id to a list of integers and add to extra
extra.extend(list(encrypted_payment_id))


print(f"Transaction extra field (extra): {extra}")

Transaction extra field (extra): [1, 2, 134, 65, 142, 182, 174, 126, 7, 146, 120, 25, 227, 171, 11, 251, 154, 237, 231, 18, 202, 2, 53, 37, 34, 233, 208, 240, 129, 223, 61, 135, 61, 2, 9, 70, 38, 80, 96, 144, 16, 51, 158]


### Summary
**What we start with:**
* Bob's Monero address: `49FgswTBtGdTZQPj7e16qc4jrWY5VuGE2aDoaMDEALknZeTy5vcjkcsCrh4deo231LKcACcjgALFji4c4SYQF5WdNCLvkN3`
* Alice's public view key: `b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867`
* Alice's public spend key: `cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37`

**What we end up with:**
* Bob's public spend key: `c95c75de8c46a69ec8b190ce1fde47165a686385910fafc69d2472ffeaddcfc3`
* Bob's public view key: `2d67eaa5f16c0c46e3052dccd0dc036f3d35bca0855f5af57e9997b63f6506bb`
* transaction key (r): `42adc98ec7a15451f3a21750b2172e14d851191a7c07d247591bbd063b698a02`
* transaction public key (rG): `0286418eb6ae7e07927819e3ab0bfb9aede712ca02352522e9d0f081df3d873d`
* encrypted payment id: `[70, 38, 80, 96, 144, 16, 51, 158]`
* stealth address (for Bob): `07ab8e6d3981d84b50f0d02ec45c2232ba239159db2f15adf44947535be13967`
* view tag (for Bob): `c6`
* stealth address (for zero change): `4e9451dd54125562c76cb59decc500b396fb2f6dc73f95876adaa6729c7db3f8`
* view tag (for zero change): `bf`
* transaction extra field (extra): `[1,2,134,65,142,182,174,126,7,146,120,25,227,171,11,251,154,237,231,18,202,2,53,37,34,233,208,240,129,223,61,135,61,2,9,70,38,80,96,144,16,51,158]`